## Python Imports

In [ ]:
!pip install accelerate
!pip install transformers
!pip install sentence-transformers
!pip install datasets
# !pip -q install google-generativeai==0.3.0
# !pip -q install google-ai-generativelanguage==0.4.0
import accelerate
import torch
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import TensorDataset
from transformers import TrainingArguments, Trainer
import matplotlib.pyplot as plt

import os
from torch.optim import AdamW
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import get_linear_schedule_with_warmup
from datasets import Dataset, DatasetDict
from accelerate import Accelerator
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# # Load the annotation file
# annotation_file = '/content/subtask-3-annotations.txt'
# text_files_dir = '/content/'

# df = pd.read_csv(annotation_file, sep='\t', header=None, names=['File Name', 'Narrative 1', 'Narrative 2', 'Explanation'])
# # Function to get the content of a file
# def get_file_content(file_name):
#     file_path = os.path.join(text_files_dir, file_name)
#     if os.path.exists(file_path):
#         with open(file_path, 'r', encoding='utf-8') as file:
#             return file.read()
#     else:
#         return "File not found"

# # Replace file names with actual file content
# df['Content'] = df['File Name'].apply(get_file_content)
# df['Prompt'] = np.where(df['Narrative 2'] != 'none',
#                         'Based on the narrative following narrative: ' + df['Narrative 2']+ '  find the summary of the: ' + df['Content'],
#                         'Based on the narrative following narrative: ' + df['Narrative 1']+ '  find the summary of the: ' + df['Content'])
# df

In [ ]:
# tedf.to_csv('testing.csv', index=False)

In [ ]:
# df.to_csv('tqsk3.csv', index=False)
df = pd.read_csv('/content/tqsk3.csv')

In [ ]:
tdf = df.iloc[:70]
vdf = df.iloc[70:77]
tedf = df.iloc[77:]

In [ ]:
print(tedf.Prompt.dtype,
tedf.Content.dtype,
tedf['Narrative 1'].dtype)
tedf['Narrative 1']

object object object


77                        URW: Speculating war outcomes
78                    CC: Criticism of climate movement
79    URW: Blaming the war on others rather than the...
80                            URW: Discrediting Ukraine
81                       CC: Downplaying climate change
82        CC: Criticism of institutions and authorities
83              URW: Negative Consequences for the West
84         CC: Questioning the measurements and science
85    URW: Blaming the war on others rather than the...
86                    URW: Amplifying war-related fears
87    URW: Blaming the war on others rather than the...
Name: Narrative 1, dtype: object

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

checkpoint = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(checkpoint)
tokenizer = BartTokenizer.from_pretrained(checkpoint)

model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [ ]:
from datasets import Dataset
import numpy as np

def tokenize_function(example):
    inputs = tokenizer(example['Prompt'], max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
    with tokenizer.as_target_tokenizer():
        targets = tokenizer(example['Explanation'], max_length=80, truncation=True, padding="max_length", return_tensors="pt")

    inputs['labels'] = targets['input_ids']
    return inputs

# df['Prompt'] = np.where(df['Narrative 2'] != 'none',
#                         'Based on the narrative following narrative: ' + df['Narrative 2']+ '  find the summary of the: ' + df['Content'],
#                         'Based on the narrative following narrative: ' + df['Narrative 1']+ '  find the summary of the: ' + df['Content'])

df_train = tdf[['Prompt', 'Explanation']].copy()
df_val = vdf[['Prompt', 'Explanation']].copy()

traindataset = Dataset.from_pandas(df_train)
train_dataset = traindataset.map(tokenize_function, batched=True)

validationdataset = Dataset.from_pandas(df_val)
validation_dataset = validationdataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/70 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

In [ ]:
! pip install rouge_score
! pip install evaluate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=dbbca40e242156412b01529fdf81965547aa9e2976e17a9a930eb616592121b5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00


In [ ]:
!pip install nltk
# NLTK imports for BLEU
from nltk.translate.bleu_score import corpus_bleu
import nltk
nltk.download('punkt_tab')
nltk.download('punkt')   # Ensure 'punkt' is downloaded
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import pandas as pd
import torch
import numpy as np

# Use evaluate instead of load_metric
from evaluate import load

# NLTK imports for BLEU
from nltk.translate.bleu_score import corpus_bleu
import nltk
nltk.download('punkt')  # Ensure 'punkt' is downloaded
from nltk.tokenize import word_tokenize

# Install 'rouge_score' outside the script using pip if needed, but not inside the script

# Load the metrics using evaluate
rouge_metric = load('rouge')

# BLEU isn't directly supported in evaluate, so we use nltk for BLEU
def compute_metrics(pred):
    pred_logits = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred_str = [tokenizer.decode(ids, skip_special_tokens=True) for ids in pred_ids]

    # Remove ignored index (-100) from labels and replace with pad token id
    labels_ids = np.where(pred.label_ids != -100, pred.label_ids, tokenizer.pad_token_id)
    label_str = [tokenizer.decode(ids, skip_special_tokens=True) for ids in labels_ids]

    # Compute ROUGE score
    rouge_output = rouge_metric.compute(predictions=pred_str, references=label_str)

    # Prepare tokens for BLEU score
    pred_tokens = [nltk.word_tokenize(text) for text in pred_str]
    label_tokens = [[nltk.word_tokenize(text)] for text in label_str]

    # Compute BLEU score using NLTK
    bleu_output = corpus_bleu(label_tokens, pred_tokens)

    # Return ROUGE and BLEU scores
    return {
        'rouge1': rouge_output['rouge1'],   # No more .mid.fmeasure, just take the value
        'rouge2': rouge_output['rouge2'],
        'rougeLsum': rouge_output['rougeLsum'],
        'bleu': bleu_output,
    }


# Initialize WandB with disabled mode
import wandb
wandb.init(mode="disabled")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',           # Directory to save checkpoints
    num_train_epochs=5,               # Number of training epochs
    per_device_train_batch_size=4,
    warmup_steps=10,                  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # Strength of weight decay
    logging_dir='./logs',             # Directory for storing logs
    logging_steps=10,                 # Log every X updates
    save_strategy="steps",            # Save the model at the end of each epoch
    eval_strategy="steps",            # Evaluate at the end of each epoch
    load_best_model_at_end=True,      # Load the best model found during evaluation
    learning_rate=2e-5,               # Learning rate
    save_total_limit=5,
    metric_for_best_model="rougeLsum",   # Use ROUGE-L to determine the best model
    greater_is_better=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
# 72ec5df7b7871e699a9c5e61b91a0cb74038b3c1

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=90, training_loss=2.5103993945651584, metrics={'train_runtime': 224.0975, 'train_samples_per_second': 1.562, 'train_steps_per_second': 0.402, 'total_flos': 758486610739200.0, 'train_loss': 2.5103993945651584, 'epoch': 5.0})

In [ ]:
model.save_pretrained("/content/")
tokenizer.save_pretrained("/content/")

('/content/tokenizer_config.json',
 '/content/special_tokens_map.json',
 '/content/vocab.json',
 '/content/merges.txt',
 '/content/added_tokens.json')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig

# generation_config = GenerationConfig(
#     # max_length=80,
#     min_length=56,
#     # early_stopping=True,
#     early_stopping=False,
#     num_beams=4,
#     length_penalty=2.0,
#     no_repeat_ngram_size=3,
#     forced_bos_token_id=0,
#     forced_eos_token_id=2
# )

# Load the model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("/content/")
tokenizer = AutoTokenizer.from_pretrained("/content/")
generation_config = GenerationConfig.from_pretrained("/content/")

/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


In [ ]:
tedf

File Name                                        Narrative 1  \
77  EN_UA_012803.txt                      URW: Speculating war outcomes   
78  EN_CC_200016.txt                  CC: Criticism of climate movement   
79  EN_UA_104523.txt  URW: Blaming the war on others rather than the...   
80  EN_UA_011260.txt                          URW: Discrediting Ukraine   
81  EN_CC_100124.txt                     CC: Downplaying climate change   
82  EN_CC_100002.txt      CC: Criticism of institutions and authorities   
83  EN_UA_101079.txt            URW: Negative Consequences for the West   
84  EN_CC_200022.txt       CC: Questioning the measurements and science   
85  EN_UA_013257.txt  URW: Blaming the war on others rather than the...   
86  EN_UA_102958.txt                  URW: Amplifying war-related fears   
87  EN_UA_027787.txt  URW: Blaming the war on others rather than the...   

                                          Narrative 2  \
77                                               none   
78                                               none   
79  URW: Blaming the war on others rather than the...   
80                                               none   
81  CC: Downplaying climate change: Ice is not mel...   
82  CC: Criticism of institutions and authorities:...   
83  URW: Negative Consequences for the West: Sanct...   
84  CC: Questioning the measurements and science: ...   
85  URW: Blaming the war on others rather than the...   
86                                               none   
87  URW: Blaming the war on others rather than the...   

                                          Explanation  \
77  Several paragraphs indicate an imminent collap...   
78  The text presents several paragraphs in which ...   
79  Throughout the text, Western countries, and sp...   
80  Throughout all the text, Ukraine is discredite...   
81  The text conveys a narrative claiming that gla...   
82  The article talks about the resistance movemen...   
83  The text presents multiple paragraphs in which...   
84  The author argues that CO2 has no effect on th...   
85  The text presents several passages that togeth...   
86  In some paragraphs, the text conveys a narrati...   
87  Throughout several paragraphs, the text sugges...   

                                              Content  \
77  Panicking Putin desperately classifies medical...   
78  Climate change protesters hoping to ‘shut down...   
79  The Lies that All Pro-U.S.-Government Media Sp...   
80  Russia has a clear plan to resolve the conflic...   
81  Glaciers Give the Lie to Global Warming Narrat...   
82  Oxford Residents Mount Resistance Against the ...   
83  German bosses, unions jointly oppose boycott o...   
84  Denmark to Punish Farmers for Cow ‘Emissions’ ...   
85  Putin says what Russia needs to do to win spec...   
86  Watch incredible first-person video as Russian...   
87  Russia Says US Is Responsible for Deadly Ukrai...   

                                               Prompt  
77  Based on the narrative following narrative: UR...  
78  Based on the narrative following narrative: CC...  
79  Based on the narrative following narrative: UR...  
80  Based on the narrative following narrative: UR...  
81  Based on the narrative following narrative: CC...  
82  Based on the narrative following narrative: CC...  
83  Based on the narrative following narrative: UR...  
84  Based on the narrative following narrative: CC...  
85  Based on the narrative following narrative: UR...  
86  Based on the narrative following narrative: UR...  
87  Based on the narrative following narrative: UR...

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


def summarize_text(text):

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    summary_ids = model.generate(
        inputs['input_ids'],
        generation_config=generation_config
        # ,
        # num_beams=4,
        # min_length=56,
        # max_length=142,
        # length_penalty=2.0,
        # early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# tedf['summary'] = tedf['Content'].apply(summarize_text)
# print(tedf['Content'])
tedf['summary'] = tedf['Prompt'].apply(summarize_text)


# def summarize_text(text):

#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
#     summary_ids = model.generate(
#         inputs['input_ids'],
#         generation_config=generation_config
#     )

#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary

# tedf['summary'] = tedf['Content'].apply(summarize_text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
<ipython-input-13-a51da2f3586b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tedf['summary'] = tedf['Prompt'].apply(summarize_text)


In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from evaluate import load

bertscore = load("bertscore")

predictions = tedf['summary'].tolist()
references = tedf['Explanation'].tolist()
# results = bertscore.compute(predictions=predictions, references=references, lang="en")
results = bertscore.compute(predictions=predictions, references=references, lang="en",  model_type="distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
results

{'precision': [0.7477627992630005,
  0.7925440073013306,
  0.8523916006088257,
  0.8063246011734009,
  0.8213151693344116,
  0.7921185493469238,
  0.7531344294548035,
  0.7989963293075562,
  0.824786365032196,
  0.8270471692085266,
  0.8040544986724854],
 'recall': [0.7949743270874023,
  0.8184242248535156,
  0.8533135056495667,
  0.8051368594169617,
  0.8297491669654846,
  0.8136792182922363,
  0.8626638650894165,
  0.817298173904419,
  0.8170541524887085,
  0.8921164274215698,
  0.8516949415206909],
 'f1': [0.7706461548805237,
  0.8052762746810913,
  0.8528522849082947,
  0.8057302832603455,
  0.8255106210708618,
  0.8027541637420654,
  0.8041868209838867,
  0.8080435991287231,
  0.8209020495414734,
  0.8583503365516663,
  0.8271893262863159],
 'hashcode': 'distilbert-base-uncased_L5_no-idf_version=0.3.12(hug_trans=4.46.2)'}

In [ ]:
predictions

["The text conveys a narrative in which Putin is accused of suffering from cancer, Parkinson's disease and other ailments as well as the effects of drugs used to treat his cancer may have distorted his mind during the war in Ukraine. The text also speculates about the possible military consequences of the situation in Russia, speculating about the potential military coups.",
 'The text criticises climate movement in several passages, accusing it of promoting corruption and promoting the destruction of the US political system. In doing so, the author criticises the political elite as well as Hollywood, Big Tech, Wall Street, Mainstream Media, and the "Deep State"',
 "The text presents several paragraphs in which the West and the US are portrayed as the aggressors in the Ukraine-Russia war, blaming the US for the coup and invasion of Ukraine, rather than Russia's invasion. The text presents a narrative in which Ukraine is accused of being the aggressor, while Russia is the victim, and th

In [ ]:
references

['Several paragraphs indicate an imminent collapse of Russian government with coup in the Kremlin to remove Putin from the leadership of the country',
 'The text presents several paragraphs in which climate activists are attacked. They are accused of being extremists attempting to disrupt public events in the US. The text also uses strong emotional words and suggests that these people should be eliminated violently',
 'Throughout the text, Western countries, and specifically the US, are blamed for being the aggressors withing the context of the Ukraine-Russia conflict. The US are accused of being behind the alleged coup happened in 2014, arguing that President Obama had the plan to attack Russia',
 'Throughout all the text, Ukraine is discredited as threatening Russian-speaking population, having a criminal government, and implement ethnic cleansing in its territories. This is accomplished through the use of one-sided reporting that gradually takes opinion form accusing Ukraine and Wes

# THIS IS ADDED LATER AFTER THE CODE WORKED SO PLS DONT CHANGE ABOVE THIS

In [ ]:
import requests

# Raw URL for the annotations file
annotations_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-3-dominant-narratives.txt"

# Load the annotations file
df = pd.read_csv(annotations_url, sep="\t", header=None, names=['article_id', 'Narrative 1', 'Narrative 2'])

# Base URL for the raw documents
base_url = "https://raw.githubusercontent.com/Khubaib2002/DATA_TASK_10/main/subtask-3-documents/"

# Function to fetch the article content from the raw documents folder
def fetch_file_content(article_id):
    file_url = f"{base_url}{article_id}"
    # print(f"Fetching URL: {file_url}")  # Debugging URL
    response = requests.get(file_url)
    # print(f"Status Code: {response.status_code}")  # Debugging response status
    if response.status_code == 200:
        return response.text
    else:
        return f"File {article_id} not found"


# Fetch article content for each article_id
df['Content'] = df['article_id'].apply(fetch_file_content)



# Generate the Prompt column
df['Prompt'] = np.where(
    df['Narrative 2'] != 'none',
    'Give the summary of the: ' + df['Content'] + 'based on' + df['Narrative 2'],
    'Give the summary of the: ' + df['Content'] + 'based on' + df['Narrative 1']
)

tedf = df.copy()

In [ ]:
print(tedf.Prompt.dtype,
tedf.Content.dtype,
tedf['Narrative 1'].dtype)


object object object


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


def summarize_text(text):

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    summary_ids = model.generate(
        inputs['input_ids'],
        generation_config=generation_config
        # ,
        # num_beams=4,
        # min_length=56,
        # max_length=142,
        # length_penalty=2.0,
        # early_stopping=True
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# tedf['summary'] = tedf['Content'].apply(summarize_text)
# print(tedf['Content'])
tedf['summary'] = tedf['Content'].apply(summarize_text)


# def summarize_text(text):

#     inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
#     summary_ids = model.generate(
#         inputs['input_ids'],
#         generation_config=generation_config
#     )

#     summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
#     return summary

# tedf['summary'] = tedf['Content'].apply(summarize_text)

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
